# Project Outline

This project looks at the stock price of Roku. By doing a time series analysis and also a sentiment analysis on social media for the stock price of Roku we will try and predict future prices.

Project flow:
 * Gather posts from twitter and reddit
 * Do a sentiment analysis for each day
 * Gather the stock price
 * Do a time series analysis on both stock price and sentiment individually
 * Combine the sentiment and stock price to try and predict future prices
 

# Packages used

In [8]:
# Installing important packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import twint
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
from psaw import PushshiftAPI
import praw
import datetime as dt
import yfinance as yf
import requests 
import requests.auth
from Config import *

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)


# This is so twint will be able to work
import nest_asyncio
nest_asyncio.apply()

# Gathering social media data

## Twitter Data

### CSV route

In [ ]:
# # Gathering twitter data

# # Configure
# config = twint.Config()
# config.Search = "$ROKU"
# config.Since = '2021-05-09'
# config.Until = '2021-05-11'
# config.Lang = 'en'
# config.Store_csv = True
# config.Min_likes = 5
# config.Limit = 20
# config.Popular_tweets = True
# config.Output = '/Users/stevenkyle/Documents/Flatiron/Capstone/Capstone'

# # Running query search
# twint.run.Search(config)


In [ ]:
# test = pd.read_csv('tweets.csv')
# test.dropna(axis=1, inplace=True)
# test.drop_duplicates(inplace=True)
# test

In [ ]:
# test.columns

### Pandas route

In [25]:
# Configure
c = twint.Config()
c.Search = '$ROKU'
c.Since = '2021-05-05'
c.Until = '2021-05-07'
c.Lang = 'en'
c.Limit = 20  #Has to be increments of 20
c.Pandas = True
c.Pandas_clean = True
c.Pandas_au = True
c.Popular_tweets = True
c.Min_likes = 10

# Running the search
twint.run.Search(c)

# Storing to dataframe
Tweets_df = twint.storage.panda.Tweets_df

# Dropping unwanted columns
Tweets_df.drop(columns=['conversation_id','created_at','place','hashtags','user_id',
                       'user_id_str','name','link','urls','photos','video','thumbnail','retweet',
                       'quote_url','near','geo','source','user_rt_id','user_rt','retweet_id','reply_to',
                       'retweet_date','translate','trans_src','trans_dest'], inplace=True)

1390048334119833601 2021-05-05 15:58:43 -0500 <ParrotStock> Whelp, Twitter is cooperating about as well as my brokerage today 😂   Going to try and post this for the 3rd time today...  No moves and trying to ignore the ST noise 😊  Despite the effort, the reds prevailed $PTON $ROKU $ATER $MGNI $IDN $SHOP $CRWD $DMTK $SKLZ  Another day 🦜  https://t.co/0lOr4sHw1l
1390047541891264516 2021-05-05 15:55:34 -0500 <StockDweebs> Day 2: Trying to get to 6/7 - figures  Starting Balance: $50,000 Unrealized P&amp;L: -$431.18 Realized P&amp;L: $0.00  Current Positions: $ROKU, $NVDA Trade Ideas: $TWTR, $BABA, $NFLX, $SQ, $SAVE, $ATVI, $SHAK  Comments, portfolio, and spreadsheets screenshots attached.  https://t.co/XMhEjSPMAc
1390029715566649346 2021-05-05 14:44:44 -0500 <StockMKTNewz> Top 10 holdings of Cathie Wood &amp; Ark Invest's $ARKK Ark Innovation ETF as of today 5/5  1 Tesla $TSLA 2 Teladoc $TDOC 3 $ROKU 4 Square $SQ 5 Shopify $SHOP 6 Zillow $Z $ZG 7 Zoom $ZM 8 Spotify $SPOT 9 Twilio $TWLO 10 E

In [26]:
Tweets_df

,id,date,timezone,tweet,language,cashtags,username,day,hour,nlikes,nreplies,nretweets,search
0,1390048334119833601,2021-05-05 15:58:43,-0500,"Whelp, Twitter is cooperating about as well as...",en,"[pton, roku, ater, mgni, idn, shop, crwd, dmtk...",ParrotStock,3,15,60,25,2,$ROKU
1,1390047541891264516,2021-05-05 15:55:34,-0500,Day 2: Trying to get to 6/7 - figures Startin...,en,"[roku, nvda, twtr, baba, nflx, sq, save, atvi,...",StockDweebs,3,15,253,33,6,$ROKU
2,1390029715566649346,2021-05-05 14:44:44,-0500,Top 10 holdings of Cathie Wood &amp; Ark Inves...,en,"[arkk, tsla, tdoc, roku, sq, shop, z, zg, zm, ...",StockMKTNewz,3,14,45,6,3,$ROKU
3,1390019124571500544,2021-05-05 14:02:39,-0500,$roku down 20% in 4 days on notbing,en,[roku],Crussian17,3,14,86,29,1,$ROKU
4,1390008384133337090,2021-05-05 13:19:58,-0500,If $roku drops tmrw on earnings im doubling do...,en,[roku],Crussian17,3,13,57,12,1,$ROKU
5,1389991777151569924,2021-05-05 12:13:59,-0500,Bought $ROKU 15OCT21 350.00c small. This is m...,en,[roku],data168,3,12,37,4,0,$ROKU
6,1389940123953991685,2021-05-05 08:48:44,-0500,@phoenixvalue $ROKU,und,[roku],WallSt_Dropout,3,08,11,1,0,$ROKU
7,1389939852368560132,2021-05-05 08:47:39,-0500,"Right now, $CMCSA is valued at 6X $ROKU... No...",en,"[cmcsa, roku, roku]",anandchokkavelu,3,08,38,4,1,$ROKU
8,1389927011745996804,2021-05-05 07:56:37,-0500,GM Stratters! Up and down we go. I prefer the ...,en,"[fdx, ma, zm, team, roku, pypl, nvda, okta, etsy]",r3dpepsi,3,07,24,4,7,$ROKU
9,1389913397026770944,2021-05-05 07:02:31,-0500,"Yellen's Error, 'Inflation Scare', Cyclical Se...",en,"[aapl, roku, snap, dash, rtx, lmt, amat, amd, ...",Sarge986,3,07,17,2,4,$ROKU


In [27]:
Tweets_df.columns

Index(['id', 'date', 'timezone', 'tweet', 'language', 'cashtags', 'username',
       'day', 'hour', 'nlikes', 'nreplies', 'nretweets', 'search'],
      dtype='object')

In [33]:
Tweets_df.iloc[1]['tweet']

'Day 2: Trying to get to 6/7 - figures  Starting Balance: $50,000 Unrealized P&amp;L: -$431.18 Realized P&amp;L: $0.00  Current Positions: $ROKU, $NVDA Trade Ideas: $TWTR, $BABA, $NFLX, $SQ, $SAVE, $ATVI, $SHAK  Comments, portfolio, and spreadsheets screenshots attached.  https://t.co/XMhEjSPMAc'

## Reddit Data

In [5]:
Reddit_subreddits = ['stocks', 'investing', 'stockmarket', 'economy', 'wallstreetbets', 'options', 'Daytrading']

### Psaw/Praw

In [58]:
r = praw.Reddit(client_id=client_id,
                client_secret=client_secret,
                user_agent=user_agent)

api = PushshiftAPI(r)

In [62]:
start_epoch=int(dt.datetime(2020, 1, 1).timestamp())
end_epoch=int(dt.datetime(2020, 2, 20).timestamp())

api_request_generator = api.search_submissions(subreddit=Reddit_subreddits,
                                               after = start_epoch, before=end_epoch,
                                               q='(ROKU)|(Roku)|(roku)|(#Roku)|(#ROKU)|(#roku)|($ROKU)|($Roku)|($roku)',
                                               limit=1000)#, score = ">50")


In [63]:
Reddit_df = pd.DataFrame(columns =['ID','Num_Comments','Score','Subreddit','Title','Upvote_Ratio','Created',
                                   'Created_utc'] )
ID=[]
Num_Comments=[]
Score=[]
Subreddit = []
Title = []
Upvote_Ratio = []
Created = []
Created_utc = []



for submissions in api_request_generator:
    ID.append(submissions.id)
    Num_Comments.append(submissions.num_comments)
    Score.append(submissions.score)
    Subreddit.append(submissions.subreddit)
    Title.append(submissions.title)
    Upvote_Ratio.append(submissions.upvote_ratio)
    Created.append(submissions.created)
    Created_utc.append(submissions.created_utc)
    
temp_df = pd.DataFrame({'ID':ID,
                        'Num_Comments':Num_Comments,
                        'Score':Score,
                        'Subreddit':Subreddit,
                        'Title':Title,
                        'Upvote_Ratio':Upvote_Ratio,
                        'Created':Created,
                        'Created_utc':Created_utc})

Reddit_df = Reddit_df.append(temp_df)
Reddit_df.sort_values(by='Score', axis=0, ascending=False, inplace=True)
Reddit_df

,ID,Num_Comments,Score,Subreddit,Title,Upvote_Ratio,Created,Created_utc
0,f6m6py,10,23,wallstreetbets,Today's lucky winners (and losers),0.91,1.582195e+09,1.582167e+09
1,f6i4ul,11,3,stocks,Is ROKU’s stock priced too high to buy given a...,0.81,1.582178e+09,1.582149e+09
2,f6ey31,0,1,wallstreetbets,I bought Roku calls.... 👍👎?,1.00,1.582166e+09,1.582137e+09
3,f6c1cm,0,1,wallstreetbets,$ROKU Calls to the moon,1.00,1.582154e+09,1.582125e+09
4,f6af7y,0,1,wallstreetbets,ROKU Calls?,1.00,1.582147e+09,1.582118e+09
...,...,...,...,...,...,...,...,...
140,ekl73k,78,31,wallstreetbets,Weekly Theta Gang Thread,0.87,1.578297e+09,1.578268e+09
141,ek440r,3,0,stocks,Why is ROKU a meme stock?,0.20,1.578208e+09,1.578179e+09
142,ek2paz,3,0,investing,Why is ROKU a meme stock?,0.25,1.578202e+09,1.578173e+09
143,ejaqzf,117,558,wallstreetbets,"Literally the first time I buy spy calls, We g...",0.98,1.578057e+09,1.578029e+09


In [53]:
Reddit_df.iloc[143]['Title']

'Literally the first time I buy spy calls, We go to war over night'

In [67]:
local_datetime_converted = dt.datetime.fromtimestamp(Reddit_df.iloc[-1]['Created_utc'])
print(local_datetime_converted)

2020-01-01 10:51:13


In [5]:
api.metadata_.get('shards')

{'failed': 0, 'skipped': 0, 'successful': 24, 'total': 24}

In [7]:
Reddit_df.shape

(100, 8)

In [8]:
Reddit_df

,ID,Num_Comments,Score,Subreddit,Title,Upvote_Ratio,Created,Created_utc
0,nkqj4f,286,3317,stocks,Amazon’s ad revenue is now twice as big as Sna...,0.97,1.621980e+09,1.621951e+09
1,ncbj4d,4,2,stocks,Want a little green mojo?,0.63,1.621036e+09,1.621007e+09
2,n99roc,29,0,stocks,Vizio Holding Corp (NASDAQ: VZIO): for those w...,0.45,1.620694e+09,1.620665e+09
3,n78n3g,4,38,stocks,"Here is a Market Recap for today Friday, May 7...",0.90,1.620449e+09,1.620421e+09
4,n6otb0,27,36,stocks,"$ROKU Crushes Earnings, Profit and Revenue Beat",0.91,1.620385e+09,1.620357e+09
5,n6iu45,21,5,stocks,How can Wall Street analysts be so wrong about...,0.73,1.620367e+09,1.620338e+09
6,n442d3,20,31,stocks,Historical Post Earnings Moves MEGA Compilatio...,0.92,1.620097e+09,1.620068e+09
7,n3z55u,9,8,stocks,What's Happening in the Markets: Week of 5/3/2021,0.76,1.620085e+09,1.620056e+09
8,n34rkx,25,223,stocks,Wall Street Week Ahead for the trading week be...,0.95,1.619985e+09,1.619956e+09
9,n2znti,21,8,stocks,"Earnings starting 5/3, good or bad?",0.91,1.619961e+09,1.619932e+09


In [9]:
#Possible data categories that can be gathered through psaw/praw

# Index(['all_awardings', 'allow_live_comments', 'author',
#        'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
#        'author_flair_type', 'author_fullname', 'author_patreon_flair',
#        'author_premium', 'awarders', 'can_mod_post', 'contest_mode',
#        'created_utc', 'domain', 'edited', 'full_link', 'gilded', 'gildings',
#        'id', 'is_crosspostable', 'is_meta', 'is_original_content',
#        'is_reddit_media_domain', 'is_robot_indexable', 'is_self', 'is_video',
#        'link_flair_background_color', 'link_flair_css_class',
#        'link_flair_richtext', 'link_flair_template_id', 'link_flair_text',
#        'link_flair_text_color', 'link_flair_type', 'locked', 'media_only',
#        'no_follow', 'num_comments', 'num_crossposts', 'over_18',
#        'parent_whitelist_status', 'permalink', 'pinned', 'post_hint',
#        'preview', 'pwls', 'retrieved_on', 'score', 'selftext', 'send_replies',
#        'spoiler', 'stickied', 'subreddit', 'subreddit_id',
#        'subreddit_subscribers', 'subreddit_type', 'thumbnail', 'title',
#        'total_awards_received', 'treatment_tags', 'upvote_ratio', 'url',
#        'whitelist_status', 'wls', 'created'],
#       dtype='object')

## Checking spacy sentiment analysis

In [2]:
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')
text = 'I had a really horrible day. It was the worst day ever! But every now and then I have a really good day that makes me happy.'
doc = nlp(text)
doc._.polarity      # Polarity: -0.125
doc._.subjectivity  # Sujectivity: 0.9
doc._.assessments  

[(['really', 'horrible'], -1.0, 1.0, None),
 (['worst', '!'], -1.0, 1.0, None),
 (['really', 'good'], 0.7, 0.6000000000000001, None),
 (['happy'], 0.8, 1.0, None)]

# Gathering stock data

In [30]:
stock_data = yf.download('ROKU','2017-01-01','2021-05-25')

[*********************100%***********************]  1 of 1 completed
